In [2]:
!pip install GEOparse


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.5/278.5 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for GEOparse: filename=GEOparse-2.0.3-py3-none-any.whl size=29046 sha256=9ff538ce56aaa8ce299dafad4b4a39aeaa83c71db3ee0a1ae53749be2b85c1ba
  Stored in directory: /root/.cache/pip/wheels/14/96/7a/2ccb3a2a9e6f57b1c758b4ced8445fad36bba011bcebb2775f
Successfully built GEOparse


In [3]:
import GEOparse
# Load the dataset
gse = GEOparse.get_GEO(geo="GSE102238")
# Access the expression data
expression_data = gse.pivot_samples('VALUE')
# Print the first few rows of the expression data
print(expression_data.head())

03-Jan-2024 19:01:06 DEBUG utils - Directory ./ already exists. Skipping.
DEBUG:GEOparse:Directory ./ already exists. Skipping.
03-Jan-2024 19:01:06 INFO GEOparse - Downloading ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE102nnn/GSE102238/soft/GSE102238_family.soft.gz to ./GSE102238_family.soft.gz
INFO:GEOparse:Downloading ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE102nnn/GSE102238/soft/GSE102238_family.soft.gz to ./GSE102238_family.soft.gz
100%|██████████| 17.9M/17.9M [00:03<00:00, 5.30MB/s]
03-Jan-2024 19:01:11 DEBUG downloader - Size validation passed
DEBUG:GEOparse:Size validation passed
03-Jan-2024 19:01:11 DEBUG downloader - Moving /tmp/tmp0_ecnet9 to /content/GSE102238_family.soft.gz
DEBUG:GEOparse:Moving /tmp/tmp0_ecnet9 to /content/GSE102238_family.soft.gz
03-Jan-2024 19:01:11 DEBUG downloader - Successfully downloaded ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE102nnn/GSE102238/soft/GSE102238_family.soft.gz
DEBUG:GEOparse:Successfully downloaded ftp://ftp.ncbi.nlm.nih.gov/geo/series/G

name             GSM2731543  GSM2731544  GSM2731545  GSM2731546  GSM2731547  \
ID_REF                                                                        
(+)E1A_r60_1         13.978      14.400      13.688      13.466      13.411   
(+)E1A_r60_a104       4.477       5.081       4.566       4.193       4.007   
(+)E1A_r60_a135       6.226       6.947       6.398       5.720       5.695   
(+)E1A_r60_a22       10.171      11.140      10.261      10.170       9.939   
(+)E1A_r60_a97        9.993      10.904       9.952       9.335       9.341   

name             GSM2731548  GSM2731549  GSM2731550  GSM2731551  GSM2731552  \
ID_REF                                                                        
(+)E1A_r60_1         14.402      13.516      14.756      14.108      14.901   
(+)E1A_r60_a104       4.186       4.411       4.737       4.336       4.447   
(+)E1A_r60_a135       5.843       6.136       6.852       6.595       6.626   
(+)E1A_r60_a22       10.497      10.028      11.360

In [7]:
X.shape

(25526, 100)

In [14]:
import numpy as np

# Specify the length
array_length = 25526

# Create an array with a repeating pattern of 1 and 0
y_array = np.tile([1, 0], array_length // 2)[:array_length]

# Print the array
print(y_array)
y_array.shape


[1 0 1 ... 0 1 0]


(25526,)

In [18]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
def monte_carlo_feature_selection(X, y, n_iterations=100, n_features_subset=10):
# Check for missing values
    missing_values = X.isnull().sum()

# Handle missing values (replace NaN with mean in this example)
    X.fillna(X.mean(), inplace=True)

    n_features = X.shape[1]
    feature_importance_scores = np.zeros(n_features)

    for _ in range(n_iterations):
        # Randomly select a subset of features
        selected_features = np.random.choice(n_features, n_features_subset, replace=False)
        X_subset = X.values[:, selected_features]

        # Split data for training and validation
        X_train, X_val, y_train, y_val = train_test_split(X_subset, y, test_size=0.3)

        # Train a classifier
        classifier = RandomForestClassifier()
        classifier.fit(X_train, y_train)

        # Update feature importance scores
        importances = classifier.feature_importances_
        for idx, feature_idx in enumerate(selected_features):
            feature_importance_scores[feature_idx] += importances[idx]

    # Normalize the feature importance scores
    feature_importance_scores /= n_iterations

    return feature_importance_scores
X=expression_data
y= y_array

importance_scores = monte_carlo_feature_selection(X, y, n_iterations=100, n_features_subset=10)

ranked_features = np.argsort(importance_scores)[::-1]  # Features ranked from most to least important
print(ranked_features)
print (importance_scores)

[99 86 33 30 97 14 49  1 41 64 54 95 43 63 53 13  8 77 50 93 45 91  7 57
 39 32 37 79 78 96 98 25  0  4 58 28 74 75 89 27 76 90 48  3 80 70 56  5
 20 18 29 34 67 11 71 81 24 23 83 68 52 69 94 42 61 36 16 26 38 72 85 17
 47 15 19 44 65  2 84 31 88 62 21  6 12 46 55 60 92 73 22 10 51 87 66 59
 35  9 82 40]
[0.01128532 0.01397872 0.00804812 0.01023202 0.01127585 0.0101893
 0.00711848 0.0122135  0.01277399 0.00386151 0.00604559 0.00978114
 0.00707089 0.0129444  0.01444851 0.00843833 0.00922127 0.00879232
 0.01008828 0.00833942 0.01017365 0.00716372 0.00613319 0.00954943
 0.00958478 0.01129885 0.00919333 0.01055406 0.01110974 0.00998772
 0.01531255 0.00778802 0.01203299 0.01599231 0.00995056 0.00412262
 0.00923861 0.01196208 0.00914211 0.0120851  0.00310006 0.01371675
 0.00926473 0.0132883  0.00825299 0.01239063 0.00694525 0.00873496
 0.01028504 0.01434611 0.01248851 0.00601302 0.00936354 0.01306038
 0.01336062 0.0064862  0.01019219 0.01211278 0.0112505  0.00438622
 0.00625224 0.00924868 0.

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Check for missing values
missing_values = X.isnull().sum()

# Handle missing values (replace NaN with mean in this example)
X.fillna(X.mean(), inplace=True)

n_features = X.shape[1]
max_features_to_test = 500

performance_tracker = []

for i in range(1, max_features_to_test + 1):
    # Select the top i features
    selected_features = ranked_features[:i]
    X_selected = X.values[:, selected_features]

    # Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)

    # Train SVM
    svm_model = SVC()
    svm_model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = svm_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Save the performance
    performance_tracker.append((i, accuracy))

    #  print out the performance of each feature set
    print(f"Number of features: {i}, Accuracy: {accuracy:.4f}")
#  Select the model with the highest accuracy
optimal_feature_count = max(performance_tracker, key=lambda x: x[1])[0]
print(f"Optimal number of features: {optimal_feature_count}")

# plot accuracy vs. number of features to manually inspect the trade-off
import matplotlib.pyplot as plt

features, accuracies = zip(*performance_tracker)
plt.plot(features, accuracies)
plt.xlabel('Number of Features')
plt.ylabel('Accuracy')
plt.title('IFS Performance')
plt.show()


Number of features: 1, Accuracy: 0.5020
Number of features: 2, Accuracy: 0.5017
Number of features: 3, Accuracy: 0.4954
Number of features: 4, Accuracy: 0.4973
Number of features: 5, Accuracy: 0.4982
Number of features: 6, Accuracy: 0.5039
Number of features: 7, Accuracy: 0.5026
Number of features: 8, Accuracy: 0.5082
Number of features: 9, Accuracy: 0.5065
Number of features: 10, Accuracy: 0.5003
Number of features: 11, Accuracy: 0.5012
Number of features: 12, Accuracy: 0.5038
Number of features: 13, Accuracy: 0.5001
Number of features: 14, Accuracy: 0.5042
Number of features: 15, Accuracy: 0.5022
Number of features: 16, Accuracy: 0.5039
Number of features: 17, Accuracy: 0.5027
Number of features: 18, Accuracy: 0.5039
Number of features: 19, Accuracy: 0.5018
Number of features: 20, Accuracy: 0.5050
Number of features: 21, Accuracy: 0.5035
Number of features: 22, Accuracy: 0.5014
Number of features: 23, Accuracy: 0.5024
Number of features: 24, Accuracy: 0.5005
Number of features: 25, A

In [ ]:
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

X = expression_data
y=[1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0]
# Check for missing values
missing_values = X.isnull().sum()

# Handle missing values (replace NaN with mean in this example)
X.fillna(X.mean(), inplace=True)

# Create a support vector machine (SVM) classifier
svm_model = SVC()

# Perform LOOCV
loo = LeaveOneOut()
scores = cross_val_score(svm_model, X.values.T, y, cv=loo)

# Calculate the overall accuracy
accuracy = scores.mean()
print(f'Overall Accuracy: {accuracy:.4f}')


Overall Accuracy: 0.8700
